# Tunning some hyperparameter to improve MAE

In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import initializers

from numpy import mean, std, asarray

from sklearn.model_selection import RepeatedKFold
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

import random

print(tf.__version__)

1.14.0


# Setting the data

Same data as previous iteration

In [4]:
inputs = pd.read_csv('input.csv')

gender = pd.get_dummies(inputs.gender, prefix='gender')
inputs = inputs.join(gender)

In [5]:
X = inputs[['age','comp_speed', 'experience', 'gender_1', 'gender_2','audr_500', 'audr_1k', 'audr_2k', 'audr_3k', 'audr_4k', 'audr_6k', 'audr_8k']].values[:-10]
x_test =  inputs[['age','comp_speed', 'experience','gender_1', 'gender_2' ,'audr_500', 'audr_1k', 'audr_2k', 'audr_3k', 'audr_4k', 'audr_6k', 'audr_8k']].values[-10:]

outputs = pd.read_csv('output.csv').iloc[:2999]
y = outputs.values[:-10]
y_test = outputs.values[-10:]

print(X.shape, y.shape)

(2989, 12) (2989, 7)


In [6]:
def evaluate_model(X, y, hyp_opt, layers_opt, neur_opt):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = try_model(n_inputs, n_outputs, hyp_opt, layers_opt, neur_opt)
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # evaluate model on test set
        mae = model.evaluate(X_test, y_test, verbose=0)
        # store result
#         print('>%.3f' % mae)
        results.append(mae)
    return results

Having a hard time installing the Keras library to add a random search (and chose to spend more time working on the model), I ended up adpting my previous function into a flexible model to try a few combination. 

In [7]:
def try_model(n_inputs, n_outputs, hyp_opt, layers_opt, neur_opt):
    model = Sequential()
    
    # Add input layer and 1st hidden layer
    model.add(Dense(neur_opt[0], input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    
    # Add additional hidden layers if necessary
    for layer in list(range(1,layers_opt)):
        model.add(Dense(neur_opt[layer], activation='relu'))
    
    #Add output layer
    model.add(Dense(n_outputs, activation='linear'))
    model.compile(loss='mae', optimizer=hyp_opt)

    return model

In [8]:
best_hyperparameters = []
combinations_to_try = 10

# Hyperparameters to randomly select from
optimizer = ['adam', 'sgd']
neurons = [6, 12, 24, 48, 64]
hidden_layers = [1,2,3]

# Try a number of combinations and save the results into a list
for iteration in range(combinations_to_try):
    hyp_opt = random.choice(optimizer)
    layers_opt = random.choice(hidden_layers)
    neur_opt = random.choices(neurons, k=layers_opt)
    
    print(f"Iteration {iteration}: Optimizer={hyp_opt}, {layers_opt} hidden_layers with {neur_opt} neurons")
    
    results = evaluate_model(X, y, hyp_opt, layers_opt, neur_opt)  
#     summarize performance
    print('    ---> MAE: %.3f (%.3f)' % (mean(results), std(results)))
    
    iteration_result = [iteration, hyp_opt, layers_opt, neur_opt, mean(results), std(results)]
    best_hyperparameters.append(iteration_result)
    
best_hyperparameters

Iteration 0: Optimizer=adam, 2 hidden_layers with [64, 12] neurons
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
    ---> MAE: 0.783 (0.061)
Iteration 1: Optimizer=adam, 3 hidden_layers with [48, 48, 12] neurons
    ---> MAE: 0.736 (0.050)
Iteration 2: Optimizer=adam, 2 hidden_layers with [48, 6] neurons
    ---> MAE: 1.030 (0.206)
Iteration 3: Optimizer=sgd, 3 hidden_layers with [64, 48, 64] neurons
    ---> MAE: 2.353 (1.020)
Iteration 4: Optimizer=sgd, 2 hidden_layers with [48, 64] neurons
    ---> MAE: 1.964 (0.496)
Iteration 5: Optimizer=sgd, 1 hidden_layers with [64] neurons
    ---> MAE: 2.863 (1.394)
Iteration 6: Optimizer=sgd, 1 hidden_layers with [48] neurons
    ---> MAE: 3.008 (1.701)
Iteration 7: Optimizer=sgd, 3 hidden_layers with [6, 48, 6] neurons
    ---> MAE: 3.330 (1.703)
Iteration 8: Optimizer=sgd, 1 hidden_layers with [48] neurons
    ---> MAE: 3.264 (2.025)
Iteration 9: Optimizer=sgd, 1 hidden

[[0, 'adam', 2, [64, 12], 0.7833956234978386, 0.060819836351644445],
 [1, 'adam', 3, [48, 48, 12], 0.7362921318924442, 0.050443015390160884],
 [2, 'adam', 2, [48, 6], 1.0299700413530828, 0.2059309614000776],
 [3, 'sgd', 3, [64, 48, 64], 2.3526620540673, 1.0199505344616553],
 [4, 'sgd', 2, [48, 64], 1.9640787284353258, 0.49562318605632283],
 [5, 'sgd', 1, [64], 2.8629847139378986, 1.3943336455625117],
 [6, 'sgd', 1, [48], 3.0076976343487325, 1.7013915927109577],
 [7, 'sgd', 3, [6, 48, 6], 3.3296448687901448, 1.7025786258219109],
 [8, 'sgd', 1, [48], 3.264220010979815, 2.025365446925021],
 [9, 'sgd', 1, [12], 2.578516405261472, 1.45696049557151]]

In [14]:
# Transform the list in a pandas dataframe to have a better vizualisation. 
results_table = pd.DataFrame.from_records(best_hyperparameters, columns=["id", "optimizer", "hidden_layers", "neurons", "mean_MAE", 'std_MAE'])
results_table.sort_values('mean_MAE')



,id,optimizer,hidden_layers,neurons,mean_MAE,std_MAE
1,1,adam,3,"[48, 48, 12]",0.736292,0.050443
0,0,adam,2,"[64, 12]",0.783396,0.060820
2,2,adam,2,"[48, 6]",1.029970,0.205931
4,4,sgd,2,"[48, 64]",1.964079,0.495623
3,3,sgd,3,"[64, 48, 64]",2.352662,1.019951
9,9,sgd,1,[12],2.578516,1.456960
5,5,sgd,1,[64],2.862985,1.394334
6,6,sgd,1,[48],3.007698,1.701392
8,8,sgd,1,[48],3.264220,2.025365
7,7,sgd,3,"[6, 48, 6]",3.329645,1.702579


In [10]:
best = results_table.loc[results_table['mean_MAE'].idxmin()][["optimizer", "hidden_layers", "neurons"]]
list(best)

['adam', 3, [48, 48, 12]]

# Select and train the best model on all the data 

In [11]:
n_inputs, n_outputs = X.shape[1], y.shape[1]

model = try_model(n_inputs, n_outputs, *list(best))

# fit the model on all data
model.fit(X, y, verbose=0, epochs=100);

In [12]:
# checks what it looks like
i=0
while i < len(x_test):
    
    row = x_test[i]
    yhat = model.predict(asarray([row]))
    print('Predicted:', np.round_(yhat[0], decimals = 2))
    print('Real value:', np.round_(y_test[i], decimals = 2))
    print('Diff:', np.round_(np.round_(yhat[0], decimals = 2) - np.round_(y_test[i], decimals = 2), decimals=2))
    print('Average MAE:', round(mean(np.round_(yhat[0], decimals = 2) - np.round_(y_test[i], decimals = 2)),2))
    print('STD MAE:', round(std(np.round_(yhat[0], decimals = 2) - np.round_(y_test[i], decimals = 2)),2))

    
    print('\n')
    i+=1

Predicted: [ 2.38  5.08  4.67  9.46 22.28 22.3  22.45]
Real value: [ 2.    6.13  4.88  9.82 22.13 21.7  20.95]
Diff: [ 0.38 -1.05 -0.21 -0.36  0.15  0.6   1.5 ]
Average MAE: 0.14
STD MAE: 0.75


Predicted: [ 2.18  5.65  4.76  7.7  15.56 15.84 16.18]
Real value: [ 2.    6.28  5.03  7.47 14.95 15.52 16.08]
Diff: [ 0.18 -0.63 -0.27  0.23  0.61  0.32  0.1 ]
Average MAE: 0.08
STD MAE: 0.38


Predicted: [1.88 2.03 1.46 1.8  2.96 2.91 2.87]
Real value: [2.   2.   2.   2.   3.   2.82 2.64]
Diff: [-0.12  0.03 -0.54 -0.2  -0.04  0.09  0.23]
Average MAE: -0.08
STD MAE: 0.23


Predicted: [1.92 1.8  3.66 4.22 5.8  5.89 6.06]
Real value: [2.   2.   3.51 4.15 5.51 5.73 6.09]
Diff: [-0.08 -0.2   0.15  0.07  0.29  0.16 -0.03]
Average MAE: 0.05
STD MAE: 0.15


Predicted: [2.   2.2  3.33 4.34 7.23 7.17 7.19]
Real value: [2.   2.   3.33 4.28 7.38 7.32 7.18]
Diff: [ 0.    0.2  -0.    0.06 -0.15 -0.15  0.01]
Average MAE: -0.0
STD MAE: 0.11


Predicted: [1.79 1.73 1.74 3.89 9.6  9.55 9.56]
Real value: [2.   